# Multiple Linear Regression

In [2]:
import csv
import pandas as pd


In [3]:
pd.set_option('display.max_columns', 65)

df = pd.read_csv('clean_data.csv', low_memory=False)


In [4]:
df.count()

parcelid                        167851
airconditioningtypeid           167851
bedroomcnt                      167851
buildingqualitytypeid           167851
calculatedbathnbr               167851
calculatedfinishedsquarefeet    167851
fips                            167851
fireplacecnt                    167851
fullbathcnt                     167851
garagecarcnt                    167851
garagetotalsqft                 167851
hashottuborspa                  167851
heatingorsystemtypeid           167851
latitude                        167851
longitude                       167851
lotsizesquarefeet               167851
poolcnt                         167851
propertycountylandusecode       167850
propertylandusetypeid           167851
rawcensustractandblock          167851
regionidcity                    164576
roomcnt                         166127
unitcnt                              5
yearbuilt                       167851
numberofstories                 167851
fireplaceflag            

In [5]:
df16 = pd.read_csv('properties_2016.csv', low_memory=False)
df_transactions16 = pd.read_csv('train_2016_v2.csv', low_memory=False)
df_merged16 = pd.merge(df16, df_transactions16, on='parcelid', how='right')
df_merged16.count()

parcelid                        90275
airconditioningtypeid           28781
architecturalstyletypeid          261
basementsqft                       43
bathroomcnt                     90275
bedroomcnt                      90275
buildingclasstypeid                16
buildingqualitytypeid           57364
calculatedbathnbr               89093
decktypeid                        658
finishedfloor1squarefeet         6856
calculatedfinishedsquarefeet    89614
finishedsquarefeet12            85596
finishedsquarefeet13               33
finishedsquarefeet15             3564
finishedsquarefeet50             6856
finishedsquarefeet6               421
fips                            90275
fireplacecnt                     9607
fullbathcnt                     89093
garagecarcnt                    29937
garagetotalsqft                 29937
hashottuborspa                   2365
heatingorsystemtypeid           56080
latitude                        90275
longitude                       90275
lotsizesquar

In [ ]:
df17 = pd.read_csv('properties_2017.csv', low_memory=False)
df_transactions17 = pd.read_csv('train_2017.csv', low_memory=False)
df_merged17 = pd.merge(df17, df_transactions17, on='parcelid', how='right')
df_merged17.count()